In [1]:
library(ggplot2)
library(pscl)
library(MASS)
library(boot)
require(qcc)

Classes and Methods for R developed in the
Political Science Computational Laboratory
Department of Political Science
Stanford University
Simon Jackman
hurdle and zeroinfl functions by Achim Zeileis



In [3]:
zinb <- read.csv("final_result.csv")
summary(zinb)

       X                 gid         sg_walk_cnt        sg_car_cnt     
 Min.   :    0   다마849996:    1   Min.   : 0.0000   Min.   : 0.0000  
 1st Qu.: 5213   다마850996:    1   1st Qu.: 0.0000   1st Qu.: 0.0000  
 Median :10426   다마851997:    1   Median : 0.0000   Median : 0.0000  
 Mean   :10426   다마851998:    1   Mean   : 0.3546   Mean   : 0.5744  
 3rd Qu.:15640   다마852998:    1   3rd Qu.: 0.0000   3rd Qu.: 0.0000  
 Max.   :20853   다마852999:    1   Max.   :12.0000   Max.   :20.0000  
                 (Other)   :20848                                      
  safe_zn_cnt         cro_cnt         spd_sign_cnt        spd_diff      
 Min.   : 0.0000   Min.   : 0.0000   Min.   : 0.0000   Min.   : 0.0000  
 1st Qu.: 0.0000   1st Qu.: 0.0000   1st Qu.: 0.0000   1st Qu.: 0.0000  
 Median : 0.0000   Median : 0.0000   Median : 0.0000   Median : 0.0000  
 Mean   : 0.2004   Mean   : 0.7236   Mean   : 0.3241   Mean   : 0.1654  
 3rd Qu.: 0.0000   3rd Qu.: 1.0000   3rd Qu.: 0.0000   3rd Qu.: 0.0000 

# 전체 교통사고

In [10]:
qcc.overdispersion.test(zinb$acci_cnt, type = "poisson")

,Obs.Var/Theor.Var,Statistic,p-value
poisson data,10.58582,220746.1,0


In [3]:
m1 <- zeroinfl(acci_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt
               + pop | sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop,
  data = zinb, dist = "negbin")
summary(m1)


Call:
zeroinfl(formula = acci_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop | sg_car_cnt + safe_zn_cnt + 
    spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb, 
    dist = "negbin")

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-0.8395 -0.2983 -0.2368 -0.2368 70.0285 

Count model coefficients (negbin with log link):
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    2.048e-01  2.624e-02   7.807 5.87e-15 ***
sg_car_cnt     1.734e-01  6.206e-03  27.938  < 2e-16 ***
safe_zn_cnt    9.041e-02  1.561e-02   5.791 7.01e-09 ***
spd_sign_cnt   2.106e-01  1.175e-02  17.929  < 2e-16 ***
spd_diff       4.971e-03  5.032e-03   0.988   0.3232    
pull_over_cnt  6.792e-01  1.515e-01   4.484 7.32e-06 ***
cctv_cnt       4.522e-01  8.271e-02   5.467 4.57e-08 ***
pop           -2.149e-04  9.074e-05  -2.368   0.0179 *  
Log(theta)    -3.305e-01  2.633e-02 -12.551  < 2e-16 ***

Zero-inflation model coe

In [4]:
m0 <- update(m1, . ~ 1)

pchisq(2 * (logLik(m1) - logLik(m0)), df = 3, lower.tail=FALSE)

'log Lik.' 0 (df=17)

In [5]:
summary(m2 <- glm.nb(acci_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb))


Call:
glm.nb(formula = acci_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb, init.theta = 0.3327584722, 
    link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.4973  -0.7020  -0.6970  -0.3583   8.8258  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -1.028e+00  1.869e-02 -55.016  < 2e-16 ***
sg_car_cnt     2.785e-01  7.940e-03  35.076  < 2e-16 ***
safe_zn_cnt    3.820e-01  2.026e-02  18.855  < 2e-16 ***
spd_sign_cnt   4.332e-01  1.522e-02  28.455  < 2e-16 ***
spd_diff      -8.420e-03  7.413e-03  -1.136  0.25600    
pull_over_cnt  1.326e+00  1.985e-01   6.681 2.38e-11 ***
cctv_cnt       3.316e-01  1.193e-01   2.781  0.00542 ** 
pop            2.898e-03  9.508e-05  30.475  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for Negative Binomial(0.3328) family taken to be 1)

    Null deviance: 20452  on 2085

In [6]:
vuong(m1, m2)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A    p-value
Raw                    28.46707 model1 > model2 < 2.22e-16
AIC-corrected          28.33263 model1 > model2 < 2.22e-16
BIC-corrected          27.79854 model1 > model2 < 2.22e-16


In [10]:
summary(m3 <- zeroinfl(acci_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop
                       | sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb))


Call:
zeroinfl(formula = acci_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop | sg_car_cnt + safe_zn_cnt + 
    spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb)

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-5.7758 -0.3616 -0.3368 -0.3368 59.6668 

Count model coefficients (poisson with log link):
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    8.820e-01  1.161e-02  75.959  < 2e-16 ***
sg_car_cnt     1.242e-01  2.009e-03  61.802  < 2e-16 ***
safe_zn_cnt    3.776e-02  4.757e-03   7.936 2.09e-15 ***
spd_sign_cnt   1.205e-01  4.158e-03  28.989  < 2e-16 ***
spd_diff       4.679e-03  1.446e-03   3.236  0.00121 ** 
pull_over_cnt  1.460e-01  3.665e-02   3.983 6.80e-05 ***
cctv_cnt       3.676e-01  1.618e-02  22.715  < 2e-16 ***
pop           -9.248e-04  4.695e-05 -19.696  < 2e-16 ***

Zero-inflation model coefficients (binomial with logit link):
                Estimate Std. Error z va

In [11]:
vuong(m1,m3)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A    p-value
Raw                    16.90952 model1 > model2 < 2.22e-16
AIC-corrected          16.90952 model1 > model2 < 2.22e-16
BIC-corrected          16.90952 model1 > model2 < 2.22e-16


In [7]:
new.d <- data.frame(zinb$sg_car_cnt,zinb$safe_zn_cnt,zinb$spd_sign_cnt,zinb$spd_diff,zinb$pull_over_cnt,zinb$cctv_cnt,zinb$pop)
names(new.d) <- c("sg_car_cnt","safe_zn_cnt","spd_sign_cnt","spd_diff","pull_over_cnt","cctv_cnt","pop")
new.d

sg_car_cnt,safe_zn_cnt,cro_cnt,spd_sign_cnt,spd_diff,pull_over_cnt,cctv_cnt,pop
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0,0,0,0,13
4,2,5,0,0,0,0,267
5,0,5,3,0,0,0,78
3,1,1,2,0,0,0,0
0,0,2,0,0,0,0,396
0,0,0,1,0,0,0,24
0,0,3,0,0,0,0,82
0,0,0,0,0,0,0,32
0,1,0,0,0,0,0,132


In [8]:
new.d$prediction <- predict(m1,newdata=new.d)

In [9]:
new.d
#prediction 열이 생김

sg_car_cnt,safe_zn_cnt,cro_cnt,spd_sign_cnt,spd_diff,pull_over_cnt,cctv_cnt,pop,prediction
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0,0,0,0,13,0.1860052
4,2,5,0,0,0,0,267,4.6496880
5,0,5,3,0,0,0,78,7.9114646
3,1,1,2,0,0,0,0,2.4527304
0,0,2,0,0,0,0,396,1.3847546
0,0,0,1,0,0,0,24,0.5481182
0,0,3,0,0,0,0,82,1.7817082
0,0,0,0,0,0,0,32,0.2751592
0,1,0,0,0,0,0,132,0.9839381


In [10]:
zinb <- cbind(zinb,new.d$prediction)
zinb

X,gid,sg_walk_cnt,sg_car_cnt,safe_zn_cnt,cro_cnt,spd_sign_cnt,spd_diff,spd_sign,pull_over_cnt,⋯,kid_pro,building_kid_cnt,building_old_cnt,acci_cnt,acci_car_cnt,acci_pm_cnt,acci_kid_cnt,acci_old_cnt,geometry,new.d$prediction
<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>
0,다바866110,0,0,0,0,0,0,0,0,⋯,0.000000,0,0,0,0,0,0,0,"MULTIPOLYGON (((127.3507618774412 36.2962226528149, 127.35076016019 36.29712419940831, 127.3518738869674 36.29712558434352, 127.3518755914034 36.29622403770472, 127.3507618774412 36.2962226528149)))",0.1860052
1,다바823157,3,4,2,5,0,0,0,0,⋯,16.479401,0,0,0,0,0,0,0,"MULTIPOLYGON (((127.3027655376963 36.33852572474733, 127.3027632646547 36.33942726265638, 127.3038775905834 36.33942909649135, 127.3038798507832 36.33852755852221, 127.3027655376963 36.33852572474733)))",4.6496880
2,다바928260,6,5,0,5,3,0,1,0,⋯,10.256410,2,0,0,0,0,0,0,"MULTIPOLYGON (((127.419673399625 36.43151969066186, 127.4196724707475 36.43242121897227, 127.4207881299057 36.43242196437237, 127.4207890458822 36.43152043603757, 127.419673399625 36.43151969066186)))",7.9114646
3,다바931203,1,3,1,1,2,0,1,0,⋯,0.000000,0,0,2,1,0,0,1,"MULTIPOLYGON (((127.4230710131166 36.38013455218083, 127.4230701251944 36.38103608833949, 127.4241850500282 36.38103680113275, 127.424185925082 36.38013526495075, 127.4230710131166 36.38013455218083)))",2.4527304
4,다바836109,2,0,0,2,0,0,0,0,⋯,7.323232,0,0,0,0,0,0,0,"MULTIPOLYGON (((127.3173525982284 36.29527473719475, 127.3173504966177 36.2961762824045, 127.318464208043 36.29617797850144, 127.3184662968392 36.29527643323608, 127.3173525982284 36.29527473719475)))",1.3847546
5,다바861174,0,0,0,0,1,0,1,0,⋯,0.000000,0,0,0,0,0,0,0,"MULTIPOLYGON (((127.3450791441312 36.35391426501025, 127.3450773577309 36.35481580264442, 127.3461919054753 36.3548172424657, 127.346193679024 36.35391570478436, 127.3450791441312 36.35391426501025)))",0.5481182
6,다바900127,0,0,0,3,0,0,0,0,⋯,0.000000,0,0,1,1,0,0,0,"MULTIPOLYGON (((127.3886063974092 36.31159021601022, 127.3886051149066 36.3124917616393, 127.3897190618861 36.31249279443445, 127.3897203315639 36.31159124877151, 127.3886063974092 36.31159021601022)))",1.7817082
7,다바948151,0,0,0,0,0,0,0,0,⋯,0.000000,9,0,0,0,0,0,0,"MULTIPOLYGON (((127.442059276762 36.33326516776619, 127.4420586091474 36.33416671136517, 127.4431728661642 36.33416724634912, 127.4431735209402 36.33326570273258, 127.442059276762 36.33326516776619)))",0.2751592
8,다바948152,0,0,1,0,0,0,0,0,⋯,0.000000,1,0,0,0,0,0,0,"MULTIPOLYGON (((127.4420586091474 36.33416671136517, 127.442057941503 36.33506825482786, 127.4431722113592 36.33506878982932, 127.4431728661642 36.33416724634912, 127.4420586091474 36.33416671136517)))",0.9839381


In [11]:
write.csv(zinb,'zinb_all_prediction.csv')

# 2.1 차-차 교통사고 - 모든 차량

In [9]:
qcc.overdispersion.test(zinb$acci_car_cnt, type = "poisson")

,Obs.Var/Theor.Var,Statistic,p-value
poisson data,9.65192,201271.5,0


In [12]:
#cro_cnt, sg_car_cnt 중 cro_cnt 빼기 (상관관계 0.61, VIF 2.03>1.91)
m1 <- zeroinfl(acci_car_cnt ~ sg_car_cnt+safe_zn_cnt+spd_sign_cnt+spd_diff+pull_over_cnt+cctv_cnt+pop
               | sg_car_cnt+safe_zn_cnt+spd_sign_cnt+spd_diff+pull_over_cnt+cctv_cnt+pop,
  data = zinb, dist = "negbin")
summary(m1)


Call:
zeroinfl(formula = acci_car_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop | sg_car_cnt + safe_zn_cnt + 
    spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb, 
    dist = "negbin")

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-0.8342 -0.3213 -0.2309 -0.2309 28.0258 

Count model coefficients (negbin with log link):
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -0.0762979  0.0290194  -2.629  0.00856 ** 
sg_car_cnt     0.1816561  0.0064167  28.310  < 2e-16 ***
safe_zn_cnt    0.1310030  0.0165128   7.933 2.13e-15 ***
spd_sign_cnt   0.2317811  0.0121289  19.110  < 2e-16 ***
spd_diff       0.0050529  0.0050804   0.995  0.31993    
pull_over_cnt  0.3881577  0.1468700   2.643  0.00822 ** 
cctv_cnt       0.4618903  0.0836696   5.520 3.38e-08 ***
pop           -0.0005185  0.0000973  -5.328 9.91e-08 ***
Log(theta)    -0.3399982  0.0292922 -11.607  < 2e-16 ***

Zero-inflation model

In [13]:
m0 <- update(m1, . ~ 1)

pchisq(2 * (logLik(m1) - logLik(m0)), df = 3, lower.tail=FALSE)

'log Lik.' 0 (df=17)

In [14]:
summary(m2 <- glm.nb(acci_car_cnt ~ sg_car_cnt+safe_zn_cnt+spd_sign_cnt+spd_diff+pull_over_cnt+cctv_cnt+pop, data = zinb))


Call:
glm.nb(formula = acci_car_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb, init.theta = 0.3491898486, 
    link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.2698  -0.6494  -0.6418  -0.6418   5.4052  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -1.2707220  0.0195429 -65.022  < 2e-16 ***
sg_car_cnt     0.2832012  0.0078562  36.048  < 2e-16 ***
safe_zn_cnt    0.4335912  0.0200043  21.675  < 2e-16 ***
spd_sign_cnt   0.4422352  0.0151119  29.264  < 2e-16 ***
spd_diff      -0.0067927  0.0072915  -0.932  0.35154    
pull_over_cnt  0.5790991  0.2002233   2.892  0.00382 ** 
cctv_cnt       0.3616833  0.1168111   3.096  0.00196 ** 
pop            0.0022297  0.0000973  22.916  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for Negative Binomial(0.3492) family taken to be 1)

    Null deviance: 19980  on 

In [15]:
vuong(m1, m2)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A    p-value
Raw                    26.24181 model1 > model2 < 2.22e-16
AIC-corrected          26.08477 model1 > model2 < 2.22e-16
BIC-corrected          25.46092 model1 > model2 < 2.22e-16


In [17]:
summary(m3 <- zeroinfl(acci_car_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop
                       | sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb))


Call:
zeroinfl(formula = acci_car_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop | sg_car_cnt + safe_zn_cnt + 
    spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb)

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-5.8454 -0.3508 -0.3183 -0.3183 22.6728 

Count model coefficients (poisson with log link):
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)    6.980e-01  1.360e-02  51.322  < 2e-16 ***
sg_car_cnt     1.290e-01  2.230e-03  57.835  < 2e-16 ***
safe_zn_cnt    4.850e-02  5.012e-03   9.677  < 2e-16 ***
spd_sign_cnt   1.287e-01  4.577e-03  28.120  < 2e-16 ***
spd_diff       4.099e-03  1.551e-03   2.643  0.00823 ** 
pull_over_cnt  5.638e-02  4.210e-02   1.339  0.18051    
cctv_cnt       3.755e-01  1.702e-02  22.055  < 2e-16 ***
pop           -1.190e-03  5.973e-05 -19.923  < 2e-16 ***

Zero-inflation model coefficients (binomial with logit link):
                Estimate Std. Error 

In [18]:
vuong(m1,m3)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A    p-value
Raw                    18.30912 model1 > model2 < 2.22e-16
AIC-corrected          18.30912 model1 > model2 < 2.22e-16
BIC-corrected          18.30912 model1 > model2 < 2.22e-16


# 2.2 차-차 교통사고 - 개인형 이동수단(pm)

In [12]:
qcc.overdispersion.test(zinb$acci_pm_cnt, type = "poisson")

,Obs.Var/Theor.Var,Statistic,p-value
poisson data,1.01041,21070.09,0.14398


In [4]:
summary(m1 <- zeroinfl(acci_pm_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt +
                       cctv_cnt + pop | sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt +
                       cctv_cnt + pop, data = zinb))


Call:
zeroinfl(formula = acci_pm_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop | sg_car_cnt + safe_zn_cnt + 
    spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb)

Pearson residuals:
     Min       1Q   Median       3Q      Max 
-0.41991 -0.07683 -0.04712 -0.04712 21.17556 

Count model coefficients (poisson with log link):
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -6.088850   0.159498 -38.175  < 2e-16 ***
sg_car_cnt     0.263702   0.051879   5.083 3.72e-07 ***
safe_zn_cnt    0.390876   0.181718   2.151   0.0315 *  
spd_sign_cnt   0.670245   0.148302   4.519 6.20e-06 ***
spd_diff      -0.115109   0.046472  -2.477   0.0133 *  
pull_over_cnt  0.337329   0.796492   0.424   0.6719    
cctv_cnt       0.943855   0.747515   1.263   0.2067    
pop            0.006224   0.001213   5.130 2.90e-07 ***

Zero-inflation model coefficients (binomial with logit link):
               Estimate Std. Error z

In [5]:
m2 <- zeroinfl(acci_pm_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop | sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop,
  data = zinb, dist = "negbin")
summary(m2)

Warning message in sqrt(diag(vc)[np]):
“NaNs produced”



Call:
zeroinfl(formula = acci_pm_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop | sg_car_cnt + safe_zn_cnt + 
    spd_sign_cnt + spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb, 
    dist = "negbin")

Pearson residuals:
     Min       1Q   Median       3Q      Max 
-0.41985 -0.07683 -0.04712 -0.04712 21.17553 

Count model coefficients (negbin with log link):
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -6.088845   0.159496 -38.176  < 2e-16 ***
sg_car_cnt     0.263716   0.051876   5.084  3.7e-07 ***
safe_zn_cnt    0.390840   0.181705   2.151   0.0315 *  
spd_sign_cnt   0.670251   0.148299   4.520  6.2e-06 ***
spd_diff      -0.115114   0.046471  -2.477   0.0132 *  
pull_over_cnt  0.336975   0.796212   0.423   0.6721    
cctv_cnt       0.943835   0.747491   1.263   0.2067    
pop            0.006224   0.001213   5.130  2.9e-07 ***
Log(theta)    13.651632         NA      NA       NA    

Zero-inflation model 

In [6]:
m0 <- update(m1, . ~ 1)

pchisq(2 * (logLik(m1) - logLik(m0)), df = 3, lower.tail=FALSE)

'log Lik.' 8.870549e-30 (df=16)

In [11]:
summary(m3 <- glm.nb(acci_pm_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt +
                       cctv_cnt + pop, data = zinb))

Warning message in glm.nb(acci_pm_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + :
“alternation limit reached”



Call:
glm.nb(formula = acci_pm_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop, data = zinb, init.theta = 0.61788983, 
    link = log)

Deviance Residuals: 
     Min        1Q    Median        3Q       Max  
-1.21160  -0.09829  -0.08516  -0.08516   2.90117  

Coefficients:
                Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -5.6166612  0.1171045 -47.963  < 2e-16 ***
sg_car_cnt     0.1699613  0.0308310   5.513 3.53e-08 ***
safe_zn_cnt    0.0712111  0.0815624   0.873  0.38262    
spd_sign_cnt   0.2010619  0.0646493   3.110  0.00187 ** 
spd_diff       0.0031463  0.0249039   0.126  0.89946    
pull_over_cnt  0.7640260  0.5184873   1.474  0.14060    
cctv_cnt      -0.2941768  0.4629365  -0.635  0.52513    
pop            0.0014606  0.0003442   4.243 2.20e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for Negative Binomial(0.6179) family taken to be 1)

    Null deviance: 112

In [8]:
vuong(m1,m2)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A p-value
Raw                  0.01087663 model1 > model2 0.49566
AIC-corrected        0.01087663 model1 > model2 0.49566
BIC-corrected        0.01087663 model1 > model2 0.49566


In [12]:
vuong(m1,m3)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A    p-value
Raw                    4.988489 model1 > model2 3.0427e-07
AIC-corrected          3.493727 model1 > model2 0.00023816
BIC-corrected         -2.444443 model2 > model1 0.00725380


# 3.1 차-사람 교통사고 - 유소년

In [7]:
qcc.overdispersion.test(zinb$acci_kid_cnt, type = "poisson")

,Obs.Var/Theor.Var,Statistic,p-value
poisson data,1.587033,33094.41,0


In [14]:
#거기에 더해 zero-inflated에 spd_diff와 kid_pro는 계산되지 않아 제외
m1 <- zeroinfl(acci_kid_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff +
               pull_over_cnt + cctv_cnt + kid_pro + building_kid_cnt | sg_car_cnt + safe_zn_cnt + spd_sign_cnt +
               pull_over_cnt + cctv_cnt + building_kid_cnt,
  data = zinb, dist = "negbin")
summary(m1)


Call:
zeroinfl(formula = acci_kid_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + kid_pro + building_kid_cnt | 
    sg_car_cnt + safe_zn_cnt + spd_sign_cnt + pull_over_cnt + cctv_cnt + 
        building_kid_cnt, data = zinb, dist = "negbin")

Pearson residuals:
     Min       1Q   Median       3Q      Max 
-0.60037 -0.12757 -0.09666 -0.09666 87.20576 

Count model coefficients (negbin with log link):
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -2.863032   0.138301 -20.701  < 2e-16 ***
sg_car_cnt        0.088688   0.020637   4.298 1.73e-05 ***
safe_zn_cnt      -0.056283   0.049275  -1.142  0.25336    
spd_sign_cnt      0.014787   0.036225   0.408  0.68314    
spd_diff          0.024798   0.012580   1.971  0.04869 *  
pull_over_cnt     0.968362   0.307883   3.145  0.00166 ** 
cctv_cnt          0.090669   0.232425   0.390  0.69646    
kid_pro           0.059635   0.006472   9.214  < 2e-16 ***
building_kid_cnt -0.0

In [15]:
m0 <- update(m1, . ~ 1)

pchisq(2 * (logLik(m1) - logLik(m0)), df = 3, lower.tail=FALSE)

'log Lik.' 2.578517e-235 (df=17)

In [16]:
summary(m2 <- glm.nb(acci_kid_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt +
                     spd_diff + pull_over_cnt + cctv_cnt + kid_pro + building_kid_cnt,
                     data = zinb))


Call:
glm.nb(formula = acci_kid_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + kid_pro + building_kid_cnt, 
    data = zinb, init.theta = 0.2003072804, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.4067  -0.2118  -0.1677  -0.1677   7.2032  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -4.2292093  0.0622812 -67.905  < 2e-16 ***
sg_car_cnt        0.2137862  0.0163786  13.053  < 2e-16 ***
safe_zn_cnt       0.0302683  0.0479942   0.631    0.528    
spd_sign_cnt      0.2250804  0.0345469   6.515 7.26e-11 ***
spd_diff          0.0004673  0.0143468   0.033    0.974    
pull_over_cnt     1.9104746  0.2890533   6.609 3.86e-11 ***
cctv_cnt         -0.2310030  0.2546114  -0.907    0.364    
kid_pro           0.0806319  0.0059214  13.617  < 2e-16 ***
building_kid_cnt  0.3111814  0.0586025   5.310 1.10e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘

In [17]:
summary(m3 <- zeroinfl(acci_kid_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt +
                     spd_diff + pull_over_cnt + cctv_cnt + kid_pro + building_kid_cnt
                       | sg_car_cnt + safe_zn_cnt + spd_sign_cnt +
                     spd_diff + pull_over_cnt + cctv_cnt + kid_pro + building_kid_cnt,
                       data = zinb))


Call:
zeroinfl(formula = acci_kid_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + kid_pro + building_kid_cnt | 
    sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + 
        cctv_cnt + kid_pro + building_kid_cnt, data = zinb)

Pearson residuals:
     Min       1Q   Median       3Q      Max 
-0.98659 -0.15129 -0.08281 -0.08281 89.23255 

Count model coefficients (poisson with log link):
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -1.018042   0.177654  -5.730 1.00e-08 ***
sg_car_cnt        0.036464   0.022401   1.628 0.103579    
safe_zn_cnt      -0.234715   0.060166  -3.901 9.57e-05 ***
spd_sign_cnt     -0.149113   0.045064  -3.309 0.000936 ***
spd_diff          0.046638   0.016480   2.830 0.004654 ** 
pull_over_cnt     1.353206   0.255391   5.299 1.17e-07 ***
cctv_cnt          0.411579   0.247732   1.661 0.096636 .  
kid_pro          -0.043155   0.009893  -4.362 1.29e-05 ***
building_kid_cnt

In [18]:
#m1 영과잉 음이항, m2 음이항
vuong(m1,m2)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A    p-value
Raw                    5.401084 model1 > model2 3.3120e-08
AIC-corrected          4.895115 model1 > model2 4.9124e-07
BIC-corrected          2.885078 model1 > model2  0.0019566


In [19]:
vuong(m1,m3)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A p-value
Raw                  -1.0088905 model2 > model1 0.15651
AIC-corrected        -0.8976197 model2 > model1 0.18469
BIC-corrected        -0.4555800 model2 > model1 0.32435


In [20]:
#m3이 영과잉 포아송
vuong(m3,m2)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A    p-value
Raw                    5.576962 model1 > model2 1.2238e-08
AIC-corrected          5.036426 model1 > model2 2.3715e-07
BIC-corrected          2.889064 model1 > model2   0.001932


# 3.2 차-사람 교통사고 - 노인

In [6]:
qcc.overdispersion.test(zinb$acci_old_cnt, type = "poisson")

,Obs.Var/Theor.Var,Statistic,p-value
poisson data,1.472823,30712.78,0


In [21]:
#VIF sg_car_cnt(1.91) < cro_cnt(2.09) -> cro_cnt 제외
m1 <- zeroinfl(acci_old_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + 
                       cctv_cnt + pop + old_pro + building_old_cnt|sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + 
                       cctv_cnt + pop + old_pro + building_old_cnt,
  data = zinb, dist = "negbin")
summary(m1)

ERROR while rich displaying an object: Error in quantile.default(x$residuals): missing values and NaN's not allowed if 'na.rm' is FALSE

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. repr::mime2repr[[mime

In [67]:
m0 <- update(m1, . ~ 1)

pchisq(2 * (logLik(m1) - logLik(m0)), df = 3, lower.tail=FALSE)

'log Lik.' 1 (df=21)

In [22]:
summary(m2 <- glm.nb(acci_old_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt +
                     spd_diff + pull_over_cnt + cctv_cnt + pop + old_pro +
                     building_old_cnt, data = zinb))


Call:
glm.nb(formula = acci_old_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop + old_pro + building_old_cnt, 
    data = zinb, init.theta = 0.3638511258, link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7504  -0.3256  -0.2231  -0.2231   4.3172  

Coefficients:
                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -3.6591917  0.0483673 -75.654  < 2e-16 ***
sg_car_cnt        0.2093225  0.0118124  17.721  < 2e-16 ***
safe_zn_cnt       0.0607846  0.0339656   1.790   0.0735 .  
spd_sign_cnt      0.2652394  0.0242366  10.944  < 2e-16 ***
spd_diff         -0.0221134  0.0109950  -2.011   0.0443 *  
pull_over_cnt     1.0524423  0.2374711   4.432 9.34e-06 ***
cctv_cnt          0.2328718  0.1534295   1.518   0.1291    
pop               0.0014664  0.0001629   9.004  < 2e-16 ***
old_pro           0.0428440  0.0024161  17.733  < 2e-16 ***
building_old_cnt  0.1938722  0.1507838   1.286   0.1985 

In [23]:
summary(m3 <- zeroinfl(acci_old_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + 
                       cctv_cnt + pop + old_pro + building_old_cnt
                       | sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + 
                       cctv_cnt + pop + old_pro + building_old_cnt, data = zinb))


Call:
zeroinfl(formula = acci_old_cnt ~ sg_car_cnt + safe_zn_cnt + spd_sign_cnt + 
    spd_diff + pull_over_cnt + cctv_cnt + pop + old_pro + building_old_cnt | 
    sg_car_cnt + safe_zn_cnt + spd_sign_cnt + spd_diff + pull_over_cnt + 
        cctv_cnt + pop + old_pro + building_old_cnt, data = zinb)

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-1.0890 -0.2646 -0.1011 -0.1011 26.7107 

Count model coefficients (poisson with log link):
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)      -1.676410   0.167522 -10.007  < 2e-16 ***
sg_car_cnt        0.053866   0.017650   3.052 0.002274 ** 
safe_zn_cnt      -0.123114   0.034259  -3.594 0.000326 ***
spd_sign_cnt      0.016185   0.023592   0.686 0.492693    
spd_diff          0.009359   0.007955   1.176 0.239410    
pull_over_cnt     0.609148   0.134996   4.512 6.41e-06 ***
cctv_cnt          0.447750   0.096696   4.631 3.65e-06 ***
pop              -0.000924   0.000214  -4.317 1.58e-05 ***
old_pro       

In [24]:
#결과에서 m1이 음이항, m2이 영과잉 포아송
vuong(m2,m3)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A    p-value
Raw                   -6.326891 model2 > model1 1.2507e-10
AIC-corrected         -5.945277 model2 > model1 1.3799e-09
BIC-corrected         -4.429260 model2 > model1 4.7278e-06
